# HW4 Trees and Forests

Official instructions:

https://www.cs.tufts.edu/cs/135/2024f/Assignments/hw4.html

This is the *starter code* notebook for Problems 1 and 2, looking at sentiment classification using bag-of-words features of product reviews.

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import time

In [2]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble

In [3]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree

ModuleNotFoundError: No module named 'pretty_print_sklearn_tree'

In [4]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')

# Load all data from train/valid/test

In [5]:
# TODO fix to path on your local system
DATA_DIR = os.path.join("/Users/student/documents/cs135/cs135-24f-assignments/hw4/data_product_reviews/")

### Load training

In [6]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))

Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [7]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))

Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [8]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))

Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [9]:
vocab_list = x_tr_df.columns.tolist()

In [10]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)

good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [11]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])

In [12]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])

In [13]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches

my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)

# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [14]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', 
    max_depth=3, min_samples_split=2, min_samples_leaf=1, 
    random_state=101)

### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [15]:
# TODO
simple_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=3, random_state=101)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [16]:
# TODO 
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)

The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['movie'] <= 0.50?
    

## 1B : Find best Decision Tree with grid search

In [17]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)

In [18]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )

**TODO Build the Grid Search** in the next coding cell

Follow the 1B instructions carefully

In [19]:
tree_grid_searcher = sklearn.model_selection.GridSearchCV(base_tree,
                                                          param_grid=tree_hyperparameter_grid_by_name,
                                                          scoring='balanced_accuracy',
                                                          cv=my_splitter,
                                                          return_train_score=True,
                                                          refit=False)

### Do the search!


In [20]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec


In [21]:
print("%.2f minutes" % (elapsed_time_sec/60))

4.95 minutes


### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [22]:
tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
print("Grid search of %3d configurations done after %6.1f sec" % (
    tree_search_results_df.shape[0], elapsed_time_sec))

Grid search of  12 configurations done after  297.0 sec


### Display search results

This block will make a pretty printed table of the results of your grid search

In [23]:
pd.set_option('display.precision', 4)
tree_keys = ['param_max_depth', 'param_min_samples_leaf']
tree_search_results_df.sort_values(tree_keys, inplace=True)
tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]

,param_max_depth,param_min_samples_leaf,mean_train_score,mean_test_score,rank_test_score,mean_fit_time
0,2,1,0.6404,0.6339,10,4.8717
1,2,3,0.6404,0.6339,10,4.8822
2,2,9,0.6404,0.6339,10,4.7894
3,8,1,0.7279,0.6989,7,14.4939
4,8,3,0.7232,0.6978,8,15.4895
5,8,9,0.7123,0.6962,9,14.3410
6,32,1,0.9187,0.7325,3,32.2808
7,32,3,0.8752,0.7349,2,33.6095
8,32,9,0.8121,0.7197,6,30.7568
9,128,1,0.9981,0.7366,1,56.1942


In [24]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")

Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 1, 'random_state': 101}


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [25]:
best_tree = base_tree.set_params(**tree_grid_searcher.best_params_) # TODO call set_params using the best_params_ found by your searcher
best_tree.fit(x_tr_NF, y_tr_N)

DecisionTreeClassifier(max_depth=128, random_state=101)

### Interpret the best decision tree

In [26]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)

The binary tree structure has 1587 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    5 are leaves
- depth   5 has   22 nodes, of which    6 are leaves
- depth   6 has   32 nodes, of which   13 are leaves
- depth   7 has   38 nodes, of which   21 are leaves
- depth   8 has   34 nodes, of which   16 are leaves
- depth   9 has   36 nodes, of which   13 are leaves
- depth  10 has   46 nodes, of which   23 are leaves
- depth  11 has   46 nodes, of which   26 are leaves
- depth  12 has   40 nodes, of which   17 are leaves
- depth  13 has   46 nodes, of which   25 are leaves
- depth  14 has   42 nodes, of which   24 are leaves
- depth  15 has   36 nodes, of which   18 are leaves
- depth  16 has   36 nodes, of which   16 are leaves
- depth  17 has   40 nodes, of which   22 are leaves
- de

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [27]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

### Fit the forest

**TODO Train on the training set** in the next coding cell

In [28]:
simple_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(max_depth=3, random_state=101)

## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [29]:
import numpy as np
import pandas as pd
import random
feat_importances = simple_forest.feature_importances_


top_indices = np.argsort(feat_importances)[::-1][:10]
top_features = [(vocab_list[i], feat_importances[i]) for i in top_indices]

zero_importance_indices = np.flatnonzero(feat_importances < 1e-5)
zero_importance_features = [(vocab_list[i], feat_importances[i]) for i in zero_importance_indices]

# Randomly select 10 features from near-zero importance set
random_indices = random.sample(range(len(zero_importance_features)), min(10, len(zero_importance_features)))
random_zero_features = [zero_importance_features[i] for i in random_indices]

# Total eligible features with near-zero importance
total_eligible = len(zero_importance_features)

# Create a table for display
left_panel = pd.DataFrame(top_features, columns=['Vocabulary Term', 'Feature Importance'])
right_panel = pd.DataFrame(random_zero_features, columns=['Vocabulary Term', 'Feature Importance'])

# Display the results
print("Highest Importance Words")
print(left_panel)

print("\n10 Random Features with Near-Zero Importance (out of {} eligible)".format(total_eligible))
print(right_panel)

Highest Importance Words
  Vocabulary Term  Feature Importance
0          return              0.0335
1           excel              0.0295
2           great              0.0293
3           worst              0.0284
4            poor              0.0269
5      disappoint              0.0244
6          i_love              0.0185
7      your_money              0.0180
8           don't              0.0176
9            bore              0.0164

10 Random Features with Near-Zero Importance (out of 7288 eligible)
  Vocabulary Term  Feature Importance
0         similar                 0.0
1         etc_etc                 0.0
2          a_pain                 0.0
3       can't_get                 0.0
4         write_a                 0.0
5        maintain                 0.0
6       getting_a                 0.0
7     way_through                 0.0
8            agre                 0.0
9         dvd_set                 0.0


In [30]:
top_indices = np.argsort(feat_importances)[::-1][:10]
top_features = [vocab_list[i] for i in top_indices]

worst_indices = np.flatnonzero(feat_importances < 1e-5)
worst_features = [vocab_list[i] for i in zero_importance_indices]
pd.DataFrame(data={"Top Features": top_features ,"Worst Features": worst_features[:10]})

,Top Features,Worst Features
0,return,good
1,excel,time
2,great,book
3,worst,i_have
4,poor,read
5,disappoint,make
6,i_love,if_you
7,your_money,thing
8,don't,and_i
9,bore,product


## 2C: Best Random Forest via grid search

Follow the instructions and using what you learn in 1B to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.)

If yours runs significantly longer, try this out on Google Colab instead.

In [31]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)


In [32]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    random_state=[101],
    )

In [33]:
# TODO construct a GridSearchCV object like you did above.

forest_grid_searcher = sklearn.model_selection.GridSearchCV(base_tree,
                                                          param_grid=forest_hyperparameter_grid_by_name,
                                                          scoring='balanced_accuracy',
                                                          cv=my_splitter,
                                                          return_train_score=True,
                                                          refit=False) 

### Do the search!

In [34]:
start_time_sec = time.time()
forest_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec

### Build dataframe of results

In [35]:
forest_search_results_df = pd.DataFrame(forest_grid_searcher.cv_results_).copy()
print("Grid search of %3d configurations done after %6.1f sec" % (
forest_search_results_df.shape[0], elapsed_time_sec))

Grid search of  10 configurations done after   12.9 sec


### Display search results

In [36]:
pd.set_option('display.precision', 4)
forest_keys = ['param_max_depth', 'param_min_samples_leaf']
forest_search_results_df.sort_values(forest_keys, inplace=True)
forest_search_results_df[forest_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]

,param_max_depth,param_min_samples_leaf,mean_train_score,mean_test_score,rank_test_score,mean_fit_time
0,16,1,0.5609,0.5332,10,0.5769
1,16,1,0.5790,0.5607,8,0.9149
2,16,1,0.6135,0.5963,6,0.9171
3,16,1,0.6427,0.6113,5,1.3469
4,16,1,0.7259,0.6857,2,1.4272
5,32,1,0.6055,0.5471,9,0.4745
6,32,1,0.6804,0.6307,4,0.6250
7,32,1,0.6803,0.5705,7,0.6830
8,32,1,0.7348,0.6756,3,1.0030
9,32,1,0.8373,0.6983,1,2.1605


### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [37]:
best_rand_forest = base_forest.set_params(**forest_grid_searcher.best_params_) # TODO call set_params using the best_params_ found by your searcher
best_rand_forest.fit(x_tr_NF, y_tr_N)

RandomForestClassifier(max_depth=32, max_features=333, random_state=101)

### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|

In [38]:
from sklearn.metrics import balanced_accuracy_score
import pandas as pd

# Define a function to evaluate a model
def evaluate_model(model, model_name, x_tr, y_tr, x_va, y_va, x_te, y_te):
    """Evaluate a model and return its performance and parameters in a dict."""
    # Predictions
    yhat_tr = model.predict(x_tr)
    yhat_va = model.predict(x_va)
    yhat_te = model.predict(x_te)
    
    # Compute balanced accuracy
    train_bacc = balanced_accuracy_score(y_tr, yhat_tr)
    valid_bacc = balanced_accuracy_score(y_va, yhat_va)
    test_bacc = balanced_accuracy_score(y_te, yhat_te)
    
    # Extract model parameters
    max_depth = getattr(model, 'max_depth', None)
    num_trees = getattr(model, 'n_estimators', 1)  # Default 1 for trees
    
    return {
        "method": model_name,
        "max depth": max_depth,
        "num trees": num_trees,
        "train BAcc": round(train_bacc, 3),
        "valid BAcc": round(valid_bacc, 3),
        "test BAcc": round(test_bacc, 3),
    }

In [39]:
models = [
    (simple_tree, "simple Tree"),
    (best_tree, "best Tree"),
    (simple_forest, "simple RandomForest"),
    (best_rand_forest, "best RandomForest"),
]

# Collect results
results = []
for model, name in models:
#     model.fit(x_tr_NF, y_tr_N)  # Ensure the model is trained
    print(name, " trained")
    results.append(evaluate_model(model, name, x_tr_NF, y_tr_N, x_va_TF, y_va_T, x_te_TF, y_te_T))


simple Tree  trained
best Tree  trained
simple RandomForest  trained
best RandomForest  trained


In [46]:
pd.DataFrame(results)

,method,max depth,num trees,train BAcc,valid BAcc,test BAcc
0,simple Tree,3,1,0.646,0.645,0.646
1,best Tree,128,1,0.998,0.737,0.726
2,simple RandomForest,3,100,0.816,0.797,0.773
3,best RandomForest,32,100,0.954,0.820,0.806
